In [1]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\Real_projects\\End_to_end_ml_project_mlops\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\Real_projects\\End_to_end_ml_project_mlops'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
# for Data validation 
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir : Path
    STATUS_FILE : str
    unzip_data_dir : Path
    all_schema : dict

In [7]:
from src.mlproject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.mlproject.utils.common import  read_yaml 
from src.mlproject.utils.common import create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema  = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE= config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema = schema
        )
        return data_validation_config

In [9]:
from src.mlproject import  logger
import os
import pandas as pd

In [10]:
class DataValidation:
    def __init__(self, config : DataValidationConfig):
        self.config = config
    
    def validate_all_columns(self) -> bool:
        try :
            validation_status = None
            
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            
            all_schema = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write('Validation status : {}'.format(validation_status))
                        
                        
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write("Validation status : {}".format(validation_status))
                        
            return validation_status
        
        except Exception as e:
            raise e

In [14]:
## data validation pipeline

try: 
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e
    

2023-09-16 19:07:46,245 - common - INFO - yaml file : {} loaded successfully
2023-09-16 19:07:46,247 - common - INFO - yaml file : {} loaded successfully
2023-09-16 19:07:46,250 - common - INFO - yaml file : {} loaded successfully
2023-09-16 19:07:46,252 - common - INFO - Created directory : artifacts
2023-09-16 19:07:46,253 - common - INFO - Created directory : artifacts/data_validation


In [12]:
df = pd.read_csv('artifacts\data_ingestion\winequality-white.csv',delimiter=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [13]:
df.columns[0]

'fixed acidity'